In [ ]:
import numpy as np
from collections import OrderedDict
import warnings

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torch.nn.functional as F

from matplotlib import pyplot as plt
import cv2

In [ ]:
# Setup torch device, using GPU if its available 
# Training with the CPU on my laptop is very very slow, so using a GPU with Google colab is preferred
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    warnings.warn("It is recommended to use train on a GPU, perhaps through Google colab, for performance")
    device = torch.device("cpu")
print(f"Using {device}")

Using cuda


In [ ]:
# Define transformations for training
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p = 0.4),
    transforms.RandomRotation(30),
    #transforms.Normalize((0.5, 0.5 ,0.5), (0.5, 0.5 ,0.5))    
])

In [ ]:
# Load in the dataset
dataset_path = "./chess_pieces"
dataset = ImageFolder(dataset_path, transform=train_transform)

train_data, val_data = torch.utils.data.random_split(
    dataset,
    [int(len(dataset)*0.8), len(dataset) - int(len(dataset)*0.8)]
)

val_data, test_data = torch.utils.data.random_split(
    val_data,
    [int(len(val_data)*0.8), len(val_data) - int(len(val_data)*0.8)]
)

train_loader = DataLoader(train_data, batch_size = 16, shuffle = True)
val_loader = DataLoader(val_data, batch_size = 16, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 1, shuffle = True)
test_loader_ordered = DataLoader(test_data, batch_size = 1, shuffle = False)

In [ ]:
# Define a neural network as a class that inherits from the torch.nn.Module class 
class ChessNet(nn.Module):
    def __init__(self):
        super(ChessNet, self).__init__()

        # use ResNet, a deep neural network model, which is particularly good for image classification
        self.model = torchvision.models.resnet50(pretrained = True)

        for parameter in self.model.parameters():
            parameter.requires_grad = False

        # Define the model of each layer TODO: is this correct?
        self.model.fc = nn.Sequential(
            nn.Linear(2048, 1000),
            nn.ReLU(),
            nn.Linear(1000, 5)
        )

    # forward propogation step TODO: is this correct? 
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
model = ChessNet() # instantiate the neural net class
learning_rate = 0.00001 # define the learning rate
# learning_rate = 0.001
epochs = 1000 # define the epochs

# use the Adam optimizer
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay = 0.001)
criterion = nn.CrossEntropyLoss() # use cross entropy loss function
min_loss = np.inf
#file = torch.load('/kaggle/working/chess.pt')
#model.load_state_dict(file)
model.to(device)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ChessNet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (

In [ ]:
# x, y = next(iter(train_loader))
# x,y = x.to(device), y.to(device)

# yhat = model(x)

# print(yhat.shape)

In [ ]:
def train(model, epochs, min_loss):
    for epoch in range(epochs):
        training_loss = 0
        model.train()
        for images, labels in train_loader:
            optimizer.zero_grad()
            images, labels = images.to(device), labels.to(device)
            yhat = model(images)
            loss = criterion(yhat, labels)
            loss.backward()
            optimizer.step()
            training_loss += loss.item()
            
            del images, labels
            torch.cuda.empty_cache()
            

        valid_loss = 0
        valid_accuracy = 0
        model.eval()
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                yhat = model(images)
                loss = criterion(yhat, labels)
                valid_loss += loss.item()
                yhat = nn.Softmax(dim = 1)(yhat)
                top_p, top_class = yhat.topk(1, dim = 1)
                num_correct = top_class == labels.view(-1,1)
                valid_accuracy += num_correct.sum().item()

                del images, labels
                torch.cuda.empty_cache()

        print("Epochs: {}.. \tTraining_loss: {:.6f}.. \tValid_loss: {:.6f}.. \tAccuracy: {:.2f}%".format(epoch, training_loss, valid_loss, (valid_accuracy/len(val_data))*100))

        if valid_loss <= min_loss:
            print("Saving Model {:.6f} ---> {:.6f}".format(min_loss, valid_loss))
            torch.save(model.state_dict(), "chess.pt")
            min_loss = valid_loss

In [ ]:
train(model, epochs, min_loss)

Epochs: 0.. 	Training_loss: 52.375653.. 	Valid_loss: 10.991291.. 	Accuracy: 25.96%
Saving Model inf ---> 10.991291
Epochs: 1.. 	Training_loss: 51.030317.. 	Valid_loss: 10.877337.. 	Accuracy: 29.81%
Saving Model 10.991291 ---> 10.877337
Epochs: 2.. 	Training_loss: 50.317967.. 	Valid_loss: 10.666012.. 	Accuracy: 29.81%
Saving Model 10.877337 ---> 10.666012
Epochs: 3.. 	Training_loss: 49.293747.. 	Valid_loss: 10.582915.. 	Accuracy: 33.65%
Saving Model 10.666012 ---> 10.582915
Epochs: 4.. 	Training_loss: 48.619225.. 	Valid_loss: 10.328755.. 	Accuracy: 37.50%
Saving Model 10.582915 ---> 10.328755
Epochs: 5.. 	Training_loss: 47.625570.. 	Valid_loss: 10.219845.. 	Accuracy: 43.27%
Saving Model 10.328755 ---> 10.219845
Epochs: 6.. 	Training_loss: 46.849006.. 	Valid_loss: 9.976274.. 	Accuracy: 50.00%
Saving Model 10.219845 ---> 9.976274
Epochs: 7.. 	Training_loss: 46.085024.. 	Valid_loss: 9.915017.. 	Accuracy: 48.08%
Saving Model 9.976274 ---> 9.915017
Epochs: 8.. 	Training_loss: 45.150654.. 	Va

In [ ]:
total_correct = 0
count = 0
classes = dataset.classes
model.load_state_dict(torch.load('./chess.pt'))
with torch.no_grad():
    model.eval()
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        yhat = model(images)
        yhat = nn.Softmax(dim = 1)(yhat)
        top_p, top_class = yhat.topk(1, dim = 1)
        eq = top_class == labels.view(-1, 1)
        # print(classes[top_class.item()])
        total_correct += eq.sum().item()
        
        if count % 1 == 0:
            print("Predicted Class: {}.. Confidence: {:.2f}%.. Ground Truth: {}".format(classes[top_class.item()], top_p.item() * 100, classes[labels.item()]))
        else:
            print(f"count%1 = {count % 1}")
        count += 1
        
print()
print()
print("Total Correct: {}/{}".format(total_correct, len(test_data)))
print("Percentage Correct: {:.2f}%..".format((total_correct/len(test_data)) * 100))
        

Predicted Class: Queen-Resized.. Confidence: 61.63%.. Ground Truth: bishop_resized
Predicted Class: Rook-resize.. Confidence: 59.63%.. Ground Truth: Rook-resize
Predicted Class: knight-resize.. Confidence: 96.19%.. Ground Truth: knight-resize
Predicted Class: bishop_resized.. Confidence: 62.09%.. Ground Truth: bishop_resized
Predicted Class: knight-resize.. Confidence: 94.50%.. Ground Truth: knight-resize
Predicted Class: knight-resize.. Confidence: 99.34%.. Ground Truth: knight-resize
Predicted Class: bishop_resized.. Confidence: 99.20%.. Ground Truth: bishop_resized
Predicted Class: knight-resize.. Confidence: 98.30%.. Ground Truth: knight-resize
Predicted Class: Rook-resize.. Confidence: 84.88%.. Ground Truth: Rook-resize
Predicted Class: Rook-resize.. Confidence: 100.00%.. Ground Truth: Rook-resize
Predicted Class: knight-resize.. Confidence: 84.41%.. Ground Truth: Queen-Resized
Predicted Class: pawn_resized.. Confidence: 97.93%.. Ground Truth: pawn_resized
Predicted Class: knight-

In [ ]:
test_images = []
test_labels = []
for images, labels in test_loader_ordered:
  images = images.to(device)
  test_images.append(images)
  labels = labels.to(device)
  test_labels.append(labels)

ind = 6
with torch.no_grad():
  model.eval()
  yp = model(test_images[ind])
  yp = nn.Softmax(dim=1)(yp)
  top_p, top_class = yp.topk(1, dim = 1)
  print(classes[top_class.item()])
  print(top_p*100)


In [ ]:
t = np.array(test_loader.dataset)
plt.imshow(np.array(t[ind][0][2]))

In [ ]:
img = cv2.imread(dataset.imgs[23][0])

In [ ]:
plt.imshow(img)

In [ ]:
img.shape